# 1. Choose a labeled text dataset used for classification

In [2]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import  balanced_accuracy_score
from sklearn.model_selection import train_test_split
df = pd.read_csv('amazon_balanced.csv')
df["Sentiment"].value_counts()   # Unbalanced dataset

Sentiment
Positive    82037
Negative    82037
Name: count, dtype: int64

In [3]:
from tabulate import tabulate
def print_relevant_words(pipe):
    words = pipe['vectorizer'].get_feature_names_out()
    print("Vocabulary size", len(words))
    coefs = pipe['model'].coef_
    classes = pipe['model'].classes_

    top_n_words = 10
    sorted_coef_indexes = coefs.argsort(axis=1)

    negative_words = [(words[i], coefs[0, i]) for i in sorted_coef_indexes[0, :top_n_words]]
    positive_words = [(words[i], coefs[0, i]) for i in sorted_coef_indexes[0, -top_n_words:]]

    table = []
    for neg, pos in zip(negative_words, positive_words):
        table.append([f"{neg[0]} | relevance: {neg[1]:.2f}", f"{pos[0]} | relevance: {pos[1]:.2f}"])

    print(tabulate(table, headers=["Negative", "Positive"], tablefmt="grid"))

# 2. Define a classification pipeline and 3. Run and evaluate the classifier

In [4]:
X = df['Review']
label_mapping = {'Positive': 1, 'Negative': 0}
y = df['Sentiment'].map(label_mapping)   # 1 for positive, 0 for negative

In [5]:
from joblib import Parallel, delayed
import numpy as np

def get_train_test_accuracy(pipeline_factory, x_vector, y_vector):
    pipeline = pipeline_factory()
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(x_vector, y_vector, test_size=0.2)
    # Train the pipeline
    pipeline.fit(X_train, y_train)
    # Predict on training set
    y_train_pred = pipeline.predict(X_train)
    train_accuracy = balanced_accuracy_score(y_train, y_train_pred)
    
    # Predict on test set
    y_test_pred = pipeline.predict(X_test)
    test_accuracy = balanced_accuracy_score(y_test, y_test_pred)

    return train_accuracy, test_accuracy

def get_mean_accurancy(pipeline_factory, x_vector, y_vector, n=10, n_jobs = -1):
    results = Parallel(n_jobs=n_jobs)(delayed(get_train_test_accuracy)(pipeline_factory, x_vector, y_vector) for i in range(n))
    results = [result for result in results if result is not None]  # Ensure results are not None

    train_accuracies = np.array([train_accuracy for train_accuracy, _ in results])
    test_accuracies = np.array([test_accuracy for _, test_accuracy in results])
    mean_train_accuracies = np.mean(train_accuracies)
    mean_test_accuracies = np.mean(test_accuracies)

    return mean_train_accuracies, mean_test_accuracies

In [ ]:
stop_words = [
"i",
"me",
"my",
"myself",
"we",
"our",
"ours",
"ourselves",
"you",
"your",
"yours",
"yourself",
"yourselves",
"he",
"him",
"his",
"himself",
"she",
"her",
"hers",
"herself",
"it",
"its",
"itself",
"they",
"them",
"their",
"theirs",
"themselves",
"what",
"which",
"who",
"whom",
"this",
"that",
"these",
"those",
"am",
"is",
"are",
"was",
"were",
"be",
"been",
"being",
"have",
"has",
"had",
"having",
"do",
"does",
"did",
"doing",
"a",
"an",
"the",
"and",
"but",
"if",
"or",
"because",
"as",
"until",
"while",
"of",
"at",
"by",
"for",
"with",
"about",
"against",
"between",
"into",
"through",
"during",
"before",
"after",
"above",
"below",
"to",
"from",
"up",
"down",
"in",
"out",
"on",
"off",
"over",
"under",
"again",
"further",
"then",
"once",
"here",
"there",
"when",
"where",
"why",
"how",
"all",
"any",
"both",
"each",
"few",
"more",
"most",
"other",
"some",
"such",
"no",
"nor",
"not",
"only",
"own",
"same",
"so",
"than",
"too",
"very",
"s",
"t",
"can",
"will",
"just",
"don",
"should",
"now",
]



In [ ]:
import re

def normal_pipeline_factory():
    return Pipeline(
        [
            (
                "vectorizer",
                CountVectorizer(
                    preprocessor=lambda x: re.sub(r"[^a-zA-Z\s]", "", x.lower()),
                    binary=True,
                    stop_words="english",
                    min_df=20,
                ),
            ),
            ("model", LogisticRegression(max_iter=1000)),
        ]
    )

pipe_count_binary = normal_pipeline_factory()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

pipe_count_binary.fit(X_train, y_train)
print("Pipe score: ", pipe_count_binary.score(X_test, y_test))
y_pred = pipe_count_binary.predict(X_test)
accuracy = balanced_accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)
print_relevant_words(pipe_count_binary)
n = 20
train_accuracy, test_accurancy = get_mean_accurancy(normal_pipeline_factory, X, y, n)
print(f"Train accuracy: {train_accuracy:.3f}, Test accuracy: {test_accurancy:.3f} over {n} runs")



In [ ]:
# Testing for arbitray text
text_positive = ["This product is unbelievably good"]
text_negative = ["This product is unbelievably bad"]  # For some reason this is classified as positive
prediction_positive = pipe_count_binary.predict(text_positive)
prediction_negative = pipe_count_binary.predict(text_negative)
print("Prediction positive: ", prediction_positive)
print("Prediction negative: ", prediction_negative)

## Lemmatized pipeline (selected as best)

In [20]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
X_lemmatized = X.apply(lambda x: ' '.join([lemmatizer.lemmatize(p) for p in x.lower().split()]))
lemmatized_stop_words = [lemmatizer.lemmatize(word) for word in stop_words]

In [21]:
import re

def lemma_pipeline_factory():
    return Pipeline(
    [
        (
            "vectorizer",
            CountVectorizer(
                preprocessor=lambda x: re.sub(r"[^a-zA-Z\s]", "", x.lower()),
                binary=True,
                stop_words=lemmatized_stop_words,
                min_df=20,
            ),
        ),
        ("model", LogisticRegression(max_iter=1000)),
    ]
)

X_train_lemmatized, X_test_lemmatized, y_train_lemmatized, y_test_lemmatized = (
    train_test_split(X_lemmatized, y, test_size=0.2, random_state=0)
)

# Print vocabulary size

pipe_lemma_count_binary = lemma_pipeline_factory()
pipe_lemma_count_binary.fit(X_train_lemmatized, y_train_lemmatized)
print(
    "Pipe score: ", pipe_lemma_count_binary.score(X_test_lemmatized, y_test_lemmatized)
)
y_pred_lemmatized = pipe_lemma_count_binary.predict(X_test_lemmatized)
accuracy = balanced_accuracy_score(y_test_lemmatized, y_pred_lemmatized)
print("Accuracy: ", accuracy)
print_relevant_words(pipe_lemma_count_binary)
n = 20
train_accuracy, test_accurancy = get_mean_accurancy(lemma_pipeline_factory, X_lemmatized, y, n)
print(f"Train accuracy: {train_accuracy:.3f}, Test accuracy: {test_accurancy:.3f} over {n} runs")

Pipe score:  0.8898369648026817
Accuracy:  0.889837507146106
Vocabulary size 9784
+----------------------------------+--------------------------------+
| Negative                         | Positive                       |
+==================================+================================+
| overrated | relevance: -3.18     | thati | relevance: 1.97        |
+----------------------------------+--------------------------------+
| dissapointing | relevance: -2.87 | hooked | relevance: 1.97       |
+----------------------------------+--------------------------------+
| boo | relevance: -2.72           | nutritionist | relevance: 1.98 |
+----------------------------------+--------------------------------+
| yuck | relevance: -2.70          | yum | relevance: 2.05          |
+----------------------------------+--------------------------------+
| yikes | relevance: -2.70         | skeptical | relevance: 2.09    |
+----------------------------------+--------------------------------+
| weakes

## Stemmed pipeline

In [47]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("english")
X_stemmed = X.apply(lambda x: ' '.join([stemmer.stem(p) for p in x.lower().split()]))
stemmed_stop_words = [stemmer.stem(word) for word in stop_words]

In [ ]:
X_train_stemmed, X_test_stemmed, y_train_stemmed, y_test_stemmed = train_test_split(
    X_stemmed, y, test_size=0.2
)

def stem_pipeline_factory():
    return Pipeline(
    [
        (
            "vectorizer",
            CountVectorizer(
                preprocessor=lambda x: re.sub(r"[^a-zA-Z\s]", "", x.lower()),
                binary=True,
                stop_words=stemmed_stop_words,
                min_df=20,
            ),
        ),
        ("model", LogisticRegression(max_iter=1000)),
    ]
)

pipe_stemmed_count = stem_pipeline_factory()
# Print vocabulary size

pipe_stemmed_count.fit(X_train_stemmed, y_train_stemmed)
print("Pipe score: ", pipe_stemmed_count.score(X_test_stemmed, y_test_stemmed))
y_pred_stemmed = pipe_stemmed_count.predict(X_test_stemmed)
accuracy = balanced_accuracy_score(y_test_stemmed, y_pred_stemmed)
print("Accuracy: ", accuracy)
print_relevant_words(pipe_stemmed_count)
n = 20
train_accuracy, test_accurancy = get_mean_accurancy(stem_pipeline_factory, X_stemmed, y, n)
print(f"Train accuracy: {train_accuracy:.3f}, Test accuracy: {test_accurancy:.3f} over {n} runs")


### Selected Pipeline: CountVectorizer, Lemmatization and Logistic Regression
#### Reasons 
* Better interpretability
* Smaller vocabulary, less dimensionality
* Small differences between classifier mean accuracy.

# 4. Assess the dataset size

In [ ]:
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline


f_list = np.logspace(0, -2.5, 50)

# Function to process one fraction f
def process_fraction(x_vector, y_vector, f, n=20):
    # Sample the dataset
    if float(f) == 1.0:
            X_sample, y_sample = x_vector, y_vector
    else:
        X_sample, _, y_sample, _ = train_test_split(x_vector, y_vector, train_size=float(f))
    train_accuracy, test_accuracy = get_mean_accurancy(lemma_pipeline_factory, X_sample, y_sample, n, n_jobs=-1)

    # Return the average accuracies for this fraction f
    print("Processed fraction", f)
    return train_accuracy, test_accuracy

def get_accuracy_sample(x_vector, y_vector, f_list):
    # Resample the dataset using .sample(frac=f), which gets a fraction of the dataset
    results = Parallel(n_jobs=-1)(delayed(process_fraction)(x_vector, y_vector, f) for f in f_list)
    
    # Separate the train and test accuracies
    train_accuracies, test_accuracies = zip(*results)
    
    return train_accuracies, test_accuracies

train_accuracies, test_accuracies = get_accuracy_sample(X_lemmatized, y, f_list)

In [ ]:
import matplotlib.pyplot as plt

error_train = [1 - x for x in train_accuracies]
error_test = [1 - x for x in test_accuracies]

plt.plot(f_list * len(df), error_train, label='Train')
plt.plot(f_list * len(df), error_test, label='Test')
plt.title("Training curves")
plt.legend()

# 5. Use topic models to refine your answers

In [8]:
def print_words_in_topics(nmf, vectorizer):
    words = vectorizer.get_feature_names_out()
    for idx, topic in enumerate(nmf.components_):
        print(f"Topic {idx}")
        for i in topic.argsort()[-5:]:
            print(words[i])
        print()
    print()

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import NMF

vectorizer_nmf_pipeline = Pipeline([
            ("vectorizer",
            CountVectorizer(
                strip_accents="ascii",
                binary=True,
                stop_words=lemmatized_stop_words,
                min_df=20,
            ),
        ),
        ('nmf', NMF(n_components=4))])
X_nmf = vectorizer_nmf_pipeline.fit_transform(X_lemmatized)

In [17]:
print_words_in_topics(vectorizer_nmf_pipeline['nmf'], vectorizer_nmf_pipeline['vectorizer'])

Topic 0
just
good
flavor
taste
like

Topic 1
ordered
amazon
box
product
wa

Topic 2
cat
day
eat
dont
food

Topic 3
food
ha
product
love
great




In [14]:
lemmatizer.lemmatize("was")

'wa'